In [ ]:
# Script: https://miyashinblog.com/ailoto6-kairyouban/
# Results: https://en.lottolyzer.com/history/new-zealand/nz-lotto/page/1/per-page/50/summary-view
%cd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install tflearn

In [ ]:
import numpy as np
import pandas as pd
import tflearn
import time
from sklearn import preprocessing
import tensorflow as tf

In [ ]:

class PredictionLSTM:

    def __init__(self):
        # LSTM訓練パラメータ設定
        self.steps_of_history = 10
        self.steps_of_furture = 10
        self.units = 5
        self.epochs = 10
        self.batch_size = 5

    # データセット作成
    def create_dataset(self, data):
        x, y = [], []
        for i in range(0, len(data) - self.steps_of_history, self.steps_of_furture):
            a = i + self.steps_of_history
            x.append(data[i:a])
            y.append(data[a])
        x = np.reshape(np.array(x), [-1, self.steps_of_history, 1])
        y = np.reshape(np.array(y), [-1, 1])
        return x, y

    # 予測用データ作成
    def create_predict_dataset(self, data):
        latest_x = np.array([data[-self.steps_of_history:]])
        latest_x = np.reshape(latest_x, (-1, self.steps_of_history, 1))
        return latest_x

    # 訓練、評価データ分割
    def split_dataset(self, x, y, test_size=0.1):
        pos = round(len(x) * (1 - test_size))
        train_x, train_y = x[:pos], y[:pos]
        test_x, test_y = x[pos:], y[pos:]
        return train_x, train_y, test_x, test_y

    # モデル作成
    def _create_model_by_tflearn(self):
        net = tflearn.input_data(shape=[None, self.steps_of_history, 1])

        # LSTM
        net = tflearn.lstm(net, n_units=self.units)

        # GRU
        # net = tflearn.gru(net, n_units=self.units)

        # GRU 複数層
        # net = tflearn.gru(net, n_units=self.units, return_seq=True)
        # net = tflearn.gru(net, n_units=self.units)
        net = tflearn.fully_connected(net, 1, activation='linear')
        net = tflearn.regression(net, optimizer='Adam', learning_rate=0.01, loss='mean_square')

        model = tflearn.DNN(net, tensorboard_verbose=0)

        return model

    # 訓練
    def train(self, train_x, train_y):
        model = self._create_model_by_tflearn()
        model.fit(train_x, train_y, validation_set=0.1, batch_size=self.batch_size, n_epoch=self.epochs)

        return model

    # 予測
    def predict(self, model, data):
        return model.predict(data)

    # 評価誤差アルゴリズム
    # RMSE(Root Mean Squared Error)
    def rmse(self, y_pred, y_true):
        return np.sqrt(((y_true - y_pred) ** 2).mean())

    # RMSLE(Root Mean Squared Logarithmic Error)
    def rmsle(self, y_pred, y_true):
        return np.sqrt(np.square(np.log(y_true + 1) - np.log(y_pred + 1)).mean())

    # MAE(Mean Absolute Error)
    def mae(self, y_pred, y_true):
        return np.mean(np.abs((y_true - y_pred)))

    # MAPE(Mean Absolute Percentage Error)
    def mape(self, y_pred, y_true):
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100



def main_process():
    START_TIME = time.time()

    dataframe = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lotto/lotto.csv')  # 学習させる過去の当選結果のパス(.csv)
    number1 = dataframe['main1']
    number2 = dataframe['main2'] - dataframe['main1']
    number3 = dataframe['main3'] - dataframe['main2']
    number4 = dataframe['main4'] - dataframe['main3']
    number5 = dataframe['main5'] - dataframe['main4']
    number6 = dataframe['main6'] - dataframe['main5']
    bonus = dataframe['bonus']

    #print(number2)

    header_name_list = [number1, number2, number3, number4, number5, number6, bonus]

    # 予測結果を最後にリストで表すための準備
    prediction_results = {}
    i = 1

    for header in header_name_list:

        # csvから読み込んだデータをint型に変換
        data = header.astype(int)
        data = data.values.reshape(data.shape[0], 1).astype(dtype=np.float32)

        # 正規化
        scaler = preprocessing.MinMaxScaler()
        data = scaler.fit_transform(data)

        # LSTMインスタンス作成
        lstm = PredictionLSTM()

        # 訓練、評価データ作成
        x, y = lstm.create_dataset(data)
        train_x, train_y, test_x, test_y = lstm.split_dataset(x, y)

        model = lstm.train(train_x, train_y)

        # 評価
        train_predict = lstm.predict(model, train_x)
        test_predict = lstm.predict(model, test_x)

        # RMSE(Root Mean Squared Error)
        train_score = lstm.rmse(train_y, train_predict)
        test_score = lstm.rmse(test_y, test_predict)
        print("Train Score: {0:.3f} RMSE".format(train_score))
        print("Test  Score: {0:.3f} RMSE".format(test_score))

        # 予測
        latest_x = lstm.create_predict_dataset(data)
        next_prediction = model.predict(latest_x)
        next_prediction = scaler.inverse_transform(next_prediction)
        print(str(header), '=', "Next prediction: {0:.0f}".format(list(next_prediction)[0][0]))

        print("Time: {0:.1f}sec".format(time.time() - START_TIME))

        if (i == 1):

            # 予測値を四捨五入して辞書に格納
            prediction_results['number1'] = int(round(next_prediction[0][0]))

        elif (i == 2):
            prediction_results['number2'] = int(round(next_prediction[0][0])) + prediction_results['number1']

        elif (i == 3):
            prediction_results['number3'] = int(round(next_prediction[0][0])) + prediction_results['number2']

        elif (i == 4):
            prediction_results['number4'] = int(round(next_prediction[0][0])) + prediction_results['number3']

        elif (i == 5):
            prediction_results['number5'] = int(round(next_prediction[0][0])) + prediction_results['number4']

        elif (i == 6):
            prediction_results['number6'] = int(round(next_prediction[0][0])) + prediction_results['number5']

        else:
            prediction_results['bonus'] = int(round(next_prediction[0][0]))

        # モデルをリセット
        # 22/2/20 エラー改修 tf. → tf.compat.v1.
        tf.compat.v1.reset_default_graph()

        i += 1

    return prediction_results

if __name__ == "__main__":
    prediction_results = main_process()
    while (prediction_results['number6'] >= 41 \
           or prediction_results['bonus'] >= 41\
        or prediction_results['bonus'] == prediction_results['number6'] \
        or prediction_results['bonus'] == prediction_results['number5'] \
        or prediction_results['bonus'] == prediction_results['number4'] \
        or prediction_results['bonus'] == prediction_results['number3'] \
        or prediction_results['bonus'] == prediction_results['number2'] \
        or prediction_results['bonus'] == prediction_results['number1'] \
        ):
        prediction_results = main_process()

print(prediction_results)